## Ch03-(3). 🎯 Autoencoder
___
* 정의
```
# 비지도학습(Unsupervised Learning) 이상탐지 영역에서 활용됨
# 오터인코더(Auto Encoder)란 입력을 출력으로 복사하는 신경망: 자기 자신을 다시 예측한다!
# 데이터를 압축하는 부분을 인코더(Encoder)
# 데이터를 복원하는 부분을 디코더(Decoder)
# 고차원 데이터를 잘 표현해주는 잠재 벡터(latent vector)을 자동으로 추출해주는 모델
# 고차원 데이터의 가장 중요한 특징을 학습하는 것
# 정상 데이터의 특징을 학습하고, 비정상 데이터를 Input하면 Loss(MSE)가 커짐
```
<img src = "https://drive.google.com/uc?id=1loR-rn4TixzAQQaepi23LDRPDS8S-0t3">
<img src = "https://drive.google.com/uc?id=1NOu1PJd1meaqNDWAL19p2f-CAg6R9k4n">
* 장단점
```
  # [good] 데이터 Label이 존재하지 않아도 사용 가능: 하지만 정상적인 데이터로 만들어진 네트워크가 있으면 몹시 좋음.
  # [good] 고차원에 데이터의 특징을 추출할 수 있음
  # [good] Auto encoder를 기반으로 다양한 알고리즘 존재
  # [bad] Hyper parameter (※ hidden layer) 설정이 어려움
  # [bad] Loss(MSE)에 대한 threshold 설정이 어려움
```
  
* 사용방법
```
# from pyod.models.auto_encoder import AutoEncoder
# 정상 Data 확보
# Hyper-paramter 결정 (※ hidden layer)
# 학습 및 예측
# threshold 및 scoring
```

* 현업사례
```
# 시계열 센서 데이터 이상 탐지
```


In [ ]:
# ▶ pyod 패키지 설치
!pip install pyod

In [ ]:
# ▶ Toy data 생성
from pyod.models.auto_encoder import AutoEncoder
from pyod.utils.data import generate_data
from pyod.utils.data import evaluate_print

contamination = 0.1  # ▶ percentage of outliers
n_train = 20000      # ▶ number of training points
n_test = 2000        # ▶ number of testing points
n_features = 300     # ▶ number of features

X_train, X_test, y_train, y_test = \
    generate_data(n_train=n_train,
                  n_test=n_test,
                  n_features=n_features,
                  contamination=contamination,
                  random_state=42)

In [ ]:
# X_train[1]
# y_train[1]
# X_train[-1]
# y_train[-1]

In [ ]:
# ▶ 모델 생성/학습 및 Hyper paramter 선정
clf_name = 'AutoEncoder'
clf = AutoEncoder(hidden_neurons =[300, 100, 100, 300], epochs=10, contamination=contamination)
clf.fit(X_train)

In [ ]:
y_train_pred = clf.labels_             # ▶ binary labels (0: inliers, 1: outliers)
y_train_scores = clf.decision_scores_  # ▶ raw outlier scores

# ▶ get the prediction on the test data
y_test_pred = clf.predict(X_test)              # ▶ outlier labels (0 or 1)
y_test_scores = clf.decision_function(X_test)  # ▶ outlier scores

In [ ]:
# ▶ score가 높을 수록 Outlier에 가까움
y_train_pred[0:5], y_train_scores[0:5]

In [ ]:
y_test_pred[1950:2000], y_test_scores[1950:2000]

In [ ]:
# ▶ Outlier 예측 데이터 수
import pandas as pd
pd.Series(y_train_pred).value_counts()

In [ ]:
# ▶ threshold 결정을 위한 modified Z점수 표준화 (※ 이상치에 Robust한 표준화 방법)
import seaborn as sns
import numpy as np

def mod_z(col) :
    med_col = col.median()
    med_abs_dev = (np.abs(col - med_col)).median()
    mod_z = 0.7413 * ((col - med_col) / med_abs_dev)
    return np.abs(mod_z)

pd_s = pd.Series(y_test_scores);
mod_z = mod_z(pd_s);
sns.distplot(mod_z);

In [ ]:
# ▶ threshold 결정을 위한 modified Z점수 표준화 (※ 이상치에 Robust한 표준화 방법)
import seaborn as sns
import numpy as np

def mod_z(col) :
    med_col = col.median()
    med_abs_dev = (np.abs(col - med_col)).median()
    mod_z = 0.7413 * ((col - med_col) / med_abs_dev)
    return np.abs(mod_z)

pd_s = pd.Series(y_test_scores);
mod_z = mod_z(pd_s);
sns.distplot(mod_z);

In [ ]:
# ▶ 성능평가
print("\nOn Training Data:")
evaluate_print(clf_name, y_train, y_train_scores)
print("\nOn Test Data:")
evaluate_print(clf_name, y_test, y_test_scores)